# 전문가를 위한 Subclassing API

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

[MNIST 데이터셋](http://yann.lecun.com/exdb/mnist/)을 로드하여 준비합니다.

In [2]:
mnist = tf.keras.datasets.mnist 

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 이를 0.0에서 1.0 사이의 값으로 정규화하여 모델이 더 잘 학습할 수 있도록 합니다.
X_train, X_test = X_train / 255.0, X_test / 255.0 

# 이미지 데이터는 보통 (높이, 너비, 채널)의 3차원 텐서로 표현됩니다. 
# mnist 데이터셋은 흑백 이미지이므로 채널이 1개이지만, 이 정보가 누락되어 2차원 텐서(높이, 너비)로 제공됩니다. 
# 따라서 채널 차원을 추가하여 데이터를 올바른 형태로 만듭니다.
X_train = X_train[..., tf.newaxis].astype("float32") 
X_test = X_test[..., tf.newaxis].astype("float32") 
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

tf.data를 사용하여 데이터셋을 섞고 배치를 만듭니다:

In [3]:
# X_train과 해당 레이블 y_train을 바탕으로 Dataset 객체를 생성
# 이 Dataset 객체는 텐서를 슬라이스하여 각 요소를 개별적으로 처리할 수 있게 해줍니다.
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

케라스(Keras)의 [모델 서브클래싱(subclassing) API](https://www.tensorflow.org/guide/keras#model_subclassing)를 사용하여 `tf.keras` 모델을 만듭니다:

In [4]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# model instance 생성
model = MyModel()

훈련에 필요한 옵티마이저(optimizer)와 손실 함수를 선택합니다: 

In [5]:
# 'from_logits=True' 매개변수는 예측값이 로짓(모델의 출력)으로 제공되며 소프트맥스 함수를 통해 확률로 변환해야 함을 나타냅니다.
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

모델의 손실과 성능을 측정할 지표를 선택합니다. 에포크가 진행되는 동안 수집된 측정 지표를 바탕으로 최종 결과를 출력합니다.

In [6]:
# 'train_loss'는 훈련 손실을 추적하기 위한 평균 메트릭입니다. 
# 이 메트릭은 각 배치의 손실을 평균하여 전체 훈련 데이터셋에 대한 평균 손실을 계산합니다.
train_loss = tf.keras.metrics.Mean(name='train_loss')

# 'train_accuracy'는 훈련 데이터의 정확도를 측정하기 위한 메트릭입니다.
# 이 메트릭은 예측된 클래스와 실제 클래스를 비교하여 정확도를 계산합니다.
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# 'test_loss'는 테스트 손실을 추적하기 위한 평균 메트릭입니다.
# 이 메트릭은 테스트 데이터셋에 대한 각 배치의 손실을 평균하여 전체 테스트 데이터셋에 대한 평균 손실을 계산합니다.
test_loss = tf.keras.metrics.Mean(name='test_loss')

# 'test_accuracy'는 테스트 데이터의 정확도를 측정하기 위한 메트릭입니다.
# 이 메트릭은 테스트 데이터셋에서 예측된 클래스와 실제 클래스를 비교하여 정확도를 계산합니다.
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

`tf.GradientTape`를 사용하여 모델을 훈련합니다:   

- with tf.GradientTape() as tape: 로 저장할 tape을 지정해주면, 이후의 GradientTape() 문맥 아래에서의 TensorFlow의 연관 연산 코드는 tape에 저장이 됩니다. 이렇게 tape에 저장된 연산 과정 (함수, 연산식) 을 가져다가 TensorFlow는 dx = tape.gradient(loss, x) 로 후진 모드 자동 미분 (Reverse mode automatic differentiation) 방법으로 손실에 대한 x의 미분을 계산합니다. 이렇게 계산한 손실에 대한 x의 미분을 역전파(backpropagation)하여 x의 값을 갱신(update)하는 작업을 반복하므로써 변수 x의 답을 찾아가는 학습을 진행합니다.   


- def 위에 @tf.fucnction annotation을 붙이면 마치 tf2.x 버전에서도 tf1.x 형태처럼 그래프 생성과 실행이 분리된 형태로 해당 함수내의 로직이 실행되게 됩니다. (실행 속도가 약간 빨라질 수 있습니다.)

In [7]:
# tf.function 데코레이터는 그래프 모드에서 실행하도록 함수를 최적화하고, 속도를 향상시킵니다.
# 이 데코레이터를 사용하면 모델의 학습 속도를 높일 수 있습니다.
@tf.function 
def train_step(images, labels):

    # tf.GradientTape는 자동 미분(주어진 입력 변수에 대한 연산의 그래디언트를 계산)을 위한 API입니다. 
    # 'with' 키워드를 사용하여 연산을 기록
    with tf.GradientTape() as tape:

        # 모델을 사용해 이미지를 예측합니다. training=True는 학습 중임을 나타냅니다.
        # Dropout과 같은 일부 레이어는 학습과 테스트 시에 다르게 동작하기 때문에 이 구분이 필요합니다.
        predictions = model(images, training=True) 

        loss = loss_object(labels, predictions) 

    # 그래디언트를 계산합니다. 
    gradients = tape.gradient(loss, model.trainable_variables) 

    # 계산된 그래디언트를 사용해 옵티마이저를 통해 학습 가능한 변수를 업데이트
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) 

    # 현재 배치에 대한 손실을 학습 손실 기록에 추가합니다.
    train_loss(loss) 

    # 현재 배치에 대한 정확도를 학습 정확도 기록에 추가합니다.
    train_accuracy(labels, predictions)

모델을 테스트합니다:

In [8]:
@tf.function 
def test_step(images, labels):

    # 모델을 사용하여 이미지를 예측합니다. training=False는 추론(또는 테스트) 중임을 나타냅니다.
    # Dropout과 같은 일부 레이어는 학습과 테스트 시에 다르게 동작하기 때문에 이 구분이 필요
    predictions = model(images, training=False) 

    t_loss = loss_object(labels, predictions) 

    # 현재 배치에 대한 손실을 테스트 손실 기록에 추가합니다.
    test_loss(t_loss) 

    # 현재 배치에 대한 정확도를 테스트 정확도 기록에 추가합니다.
    test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5 

for epoch in range(EPOCHS):

    # 각 에폭이 시작될 때 마다, 학습과 테스트의 손실 및 정확도 메트릭을 리셋
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    # 학습 데이터셋의 모든 배치에 대해 한 단계의 학습을 수행
    for images, labels in train_ds:
        train_step(images, labels)

    # 테스트 데이터셋의 모든 배치에 대해 한 단계의 테스트를 수행
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    # 각 에폭이 끝날 때마다 학습 손실, 학습 정확도, 테스트 손실, 테스트 정확도를 출력
    print(
        f'Epoch {epoch + 1}, ' 
        f'Loss: {train_loss.result():.4f}, ' 
        f'Accuracy: {train_accuracy.result() * 100:.4f}, ' 
        f'Test Loss: {test_loss.result():.4f}, ' 
        f'Test Accuracy: {test_accuracy.result() * 100:.4f}'
    )

Epoch 1, Loss: 0.1337, Accuracy: 96.0050, Test Loss: 0.0583, Test Accuracy: 98.1000
Epoch 2, Loss: 0.0398, Accuracy: 98.7450, Test Loss: 0.0650, Test Accuracy: 97.7600
Epoch 3, Loss: 0.0211, Accuracy: 99.3067, Test Loss: 0.0553, Test Accuracy: 98.3400
Epoch 4, Loss: 0.0128, Accuracy: 99.5567, Test Loss: 0.0696, Test Accuracy: 98.2200
Epoch 5, Loss: 0.0090, Accuracy: 99.7050, Test Loss: 0.0620, Test Accuracy: 98.2400
